In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Корак 1: Дефинишите Pydantic моделе за структурисане излазе

Ови модели дефинишу **шему** коју ће агенти враћати. Коришћење `response_format` са Pydantic обезбеђује:
- ✅ Типски сигурно извлачење података
- ✅ Аутоматска валидација
- ✅ Без грешака у парсирању одговорa у слободном тексту
- ✅ Лако условно рутирање на основу поља


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Корак 2: Направите алат за резервацију хотела

Овај алат ће користити **availability_agent** да провери да ли су собе доступне. Користимо декоратор `@ai_function` да:
- Претворимо Python функцију у алат који може да позове AI
- Аутоматски генеришемо JSON шему за LLM
- Обрадимо валидацију параметара
- Омогућимо аутоматско позивање од стране агената

За овај пример:
- **Стокхолм, Сијетл, Токио, Лондон, Амстердам** → Имају доступне собе ✅
- **Сви остали градови** → Немају доступне собе ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Корак 3: Дефинишите функције услова за рутирање

Ове функције анализирају одговор агента и одређују којим путем ће се кретати у радном процесу.

**Кључни образац:**
1. Проверите да ли је порука `AgentExecutorResponse`
2. Парсирајте структуриран излаз (Pydantic модел)
3. Вратите `True` или `False` за контролу рутирања

Радни процес ће процењивати ове услове на **ивицама** како би одлучио који извршилац ће бити позван следећи.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Корак 4: Креирање прилагођеног извршиоца за приказ

Извршиоци су компоненте радног процеса које обављају трансформације или споредне ефекте. Користимо декоратор `@executor` да креирамо прилагођеног извршиоца који приказује коначни резултат.

**Кључни концепти:**
- `@executor(id="...")` - Региструје функцију као извршиоца радног процеса
- `WorkflowContext[Never, str]` - Типске назнаке за улаз/излаз
- `ctx.yield_output(...)` - Испоручује коначни резултат радног процеса


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Корак 5: Учитавање променљивих окружења

Конфигуришите LLM клијента. Овај пример ради са:
- **GitHub моделима** (бесплатан ниво са GitHub токеном)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Корак 6: Креирање AI агената са структурираним излазима

Креирамо **три специјализована агента**, сваки обавијен у `AgentExecutor`:

1. **availability_agent** - Проверава доступност хотела користећи алат
2. **alternative_agent** - Предлаже алтернативне градове (када нема слободних соба)
3. **booking_agent** - Подстиче резервацију (када су собе доступне)

**Кључне карактеристике:**
- `tools=[hotel_booking]` - Омогућава алат агенту
- `response_format=PydanticModel` - Присиљава структурирани JSON излаз
- `AgentExecutor(..., id="...")` - Обавија агента за употребу у радном процесу


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Корак 7: Изградите ток рада са условним везама

Сада користимо `WorkflowBuilder` за конструисање графа са условним рутирањем:

**Структура тока рада:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Кључне методе:**
- `.set_start_executor(...)` - Поставља почетну тачку
- `.add_edge(from, to, condition=...)` - Додаје условну везу
- `.build()` - Финализује ток рада


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Корак 8: Покрените тест случај 1 - Град БЕЗ доступности (Париз)

Хајде да тестирамо пут **без доступности** тако што ћемо затражити хотеле у Паризу (који нема собе у нашој симулацији).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Корак 9: Покрените тест случај 2 - Град СА доступношћу (Стокхолм)

Сада ћемо тестирати пут **доступности** тако што ћемо затражити хотеле у Стокхолму (који има собе у нашој симулацији).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Кључни закључци и наредни кораци

### ✅ Шта сте научили:

1. **WorkflowBuilder образац**
   - Користите `.set_start_executor()` за дефинисање почетне тачке
   - Користите `.add_edge(from, to, condition=...)` за условно рутирање
   - Позовите `.build()` да финализујете ток рада

2. **Условно рутирање**
   - Функције за услове анализирају `AgentExecutorResponse`
   - Парсирајте структурисане излазе за доношење одлука о рутирању
   - Вратите `True` да активирате везу, `False` да је прескочите

3. **Интеграција алата**
   - Користите `@ai_function` за претварање Python функција у AI алате
   - Агенти аутоматски позивају алате када је потребно
   - Алати враћају JSON који агенти могу да парсирају

4. **Структурисани излази**
   - Користите Pydantic моделе за типски сигурно извлачење података
   - Поставите `response_format=MyModel` приликом креирања агената
   - Парсирајте одговоре помоћу `Model.model_validate_json()`

5. **Прилагођени извршиоци**
   - Користите `@executor(id="...")` за креирање компоненти токова рада
   - Извршиоци могу трансформисати податке или извршавати споредне ефекте
   - Користите `ctx.yield_output()` за производњу резултата токова рада

### 🚀 Примена у стварном свету:

- **Резервација путовања**: Провера доступности, предлози алтернатива, поређење опција
- **Корисничка подршка**: Рутирање на основу типа проблема, сентимента, приоритета
- **Е-трговина**: Провера залиха, предлози алтернатива, обрада наруџбина
- **Модерација садржаја**: Рутирање на основу токсичности, корисничких пријава
- **Токови одобрења**: Рутирање на основу износа, улоге корисника, нивоа ризика
- **Вишестепена обрада**: Рутирање на основу квалитета и потпуности података

### 📚 Наредни кораци:

- Додајте сложеније услове (више критеријума)
- Имплементирајте петље уз управљање стањем токова рада
- Додајте под-токове за поновно употребљиве компоненте
- Интегришите са стварним API-јевима (резервација хотела, системи залиха)
- Додајте обраду грешака и резервне путеве
- Визуализујте токове рада помоћу уграђених алата за визуализацију



---

**Одрицање од одговорности**:  
Овај документ је преведен помоћу услуге за превођење уз помоћ вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људског преводиоца. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
